In [3]:
using Pkg
Pkg.activate("."); Pkg.instantiate()

  Activating project at `~/Documents/GitHub/AMO-Single-Project`


In [4]:
#import Pkg
Pkg.add("AxisArrays")
Pkg.add("JuMP")
Pkg.add("XLSX")
Pkg.add("DataFrames")
Pkg.add("Plots")
Pkg.add("AxisArrays")
Pkg.add("Statistics")
Pkg.add("PGFPlotsX")
Pkg.add("AxisArrays")
Pkg.add("CPLEX")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/AMO-Single-Project/Project.toml`
  No Changes to `~/Documents/GitHub/AMO-Single-Project/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/AMO-Single-Project/Project.toml`
  No Changes to `~/Documents/GitHub/AMO-Single-Project/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/AMO-Single-Project/Project.toml`
  No Changes to `~/Documents/GitHub/AMO-Single-Project/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/AMO-Single-Project/Project.toml`
  No Changes to `~/Documents/GitHub/AMO-Single-Project/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/AMO-Single-Project/Project.toml`
  No Changes to `~/Documents/GitHub/AMO-Single-Project/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/AMO-Single-

In [5]:
using JuMP, XLSX, DataFrames, Plots, AxisArrays, Statistics, PGFPlotsX, CPLEX

[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]


In [6]:
pgfplotsx()

Plots.PGFPlotsXBackend()

# Data
Read in data.

xl = XLSX.readxlsx("C2_Bilevel_Demand_Response.xlsx") 
data = Dict()

for s in XLSX.sheetnames(xl)
    data[s] = DataFrame(XLSX.gettable(xl[s]; infer_eltypes=true))
end

Define sets and cardinalities from data.

G = data["unitinfo"][:, :ID]
L = data["lineinfo"][:, :ID]
T = data["load"][:, :t]
N = data["nodeinfo"][:, :ID]
S = unique(data["scenarios"][:, :major])
K = unique(data["DRCinfo"][:, :k])

NG = length(G)
NL = length(L)
NT = length(T)
NN = length(N)
NS = length(S)
NK = length(K)

# Workshop Test

#Generator (unit) data.
gd = Dict(c => AxisArray(data["unitinfo"][:, c], g=G) for c in names(data["unitinfo"]));

#Line data
ld = Dict(c => AxisArray(data["lineinfo"][:, c], l=L) for c in names(data["lineinfo"]));

B = AxisArray(zeros(NL, NN), l=L, n=N)
for l in L 
    for (idn, n) in enumerate(N)
        if ld["from"][l] == idn
            B[l, n] = -1/ld["X"][l]
        elseif ld["to"][l] == idn
            B[l, n] = 1/ld["X"][l]
        end
    end
end

#Load data 
load = AxisArray(Matrix(data["load"][:, Not(:t)]), t=T, n=N);
cost_ls = 100.0

#Wind Scenarios.
wind_cap = AxisArray(zeros(NN), n=N)
wind_cap["n6"] = 20.0

pi = AxisArray(ones(NS)/NS, s=S)

wind = AxisArray(zeros(NS, NN, NT), s=S, n=N, t=T)
for r in eachrow(data["scenarios"])
    for t in T
        wind[r[:major], r[:minor], t] = r[t]*wind_cap[r[:minor]]
    end
end

ewind = mean(wind, dims=1)[1, :, :]
stdwind = std(wind, dims=1)[1, :, :]
cost_ws = 20.0

#DRCinfo
cols = names(data["DRCinfo"][:, Not(["t", "n", "k"])])

DRCd = Dict(c => AxisArray(zeros(NT, NN, NK), t=T, n=N, k=K) for c in cols)

for r in eachrow(data["DRCinfo"])
    for c in cols
        DRCd[c][r[:t], r[:n], r[:k]] = r[c]
    end
end

#DRSinfo
cols = names(data["DRSinfo"][:, Not(["t", "n", "k"])])

DRSd = Dict(c => AxisArray(zeros(NT, NN, NK), t=T, n=N, k=K) for c in cols)

for r in eachrow(data["DRSinfo"])
    for c in cols
        DRSd[c][r[:t], r[:n], r[:k]] = r[c]
    end
end

max_rec = 2.0

#Node info
ng = AxisArray(
    [[g for g in G if gd["origin"][g] == idn] for (idn, n) in enumerate(N)],
    n = N
)

nl = AxisArray(zeros(Int, NN, NL), n=N, l=L)
for l in L
    nl[N[ld["from"][l]], l] = 1
    nl[N[ld["to"][l]], l] = -1
end


# Read In And Prepare Data 

In [7]:
xl = XLSX.readxlsx("data/curtailmentinfo.xlsx")
data_lina = Dict{String, DataFrame}()
data_lina["curtailmentinfo"] = DataFrame(XLSX.gettable(xl["curtailmentinfo"]; infer_eltypes=true))

xl = XLSX.readxlsx("data/generatorinfo.xlsx")
#generatorinfo = XLSX.sheetnames(xl)
data_lina["generatorinfo"] = DataFrame(XLSX.gettable(xl["generatorinfo"]; infer_eltypes=true))

xl = XLSX.readxlsx("data/initdemand.xlsx")
initdemand = XLSX.sheetnames(xl)
data_lina["initdemand"] = DataFrame(XLSX.gettable(xl["initdemand"]; infer_eltypes=true))

xl = XLSX.readxlsx("data/lineconstraints.xlsx")
#linecontraints = XLSX.sheetnames(xl)
data_lina["lineconstraints"] = DataFrame(XLSX.gettable(xl["lineconstraints"]; infer_eltypes=true))

xl = XLSX.readxlsx("data/nodeinfo.xlsx")
nodeinfo = XLSX.sheetnames(xl)
data_lina["nodeinfo"] = DataFrame(XLSX.gettable(xl["nodeinfo"]; infer_eltypes=true))

xl = XLSX.readxlsx("data/shiftinginfo.xlsx")
shiftinginfo = XLSX.sheetnames(xl)
data_lina["shiftinginfo"] = DataFrame(XLSX.gettable(xl["shiftinginfo"]; infer_eltypes=true))

xl = XLSX.readxlsx("data/windfarminfo.xlsx")
windfarminfo = XLSX.sheetnames(xl)
data_lina["windfarminfo"] = DataFrame(XLSX.gettable(xl["windfarminfo"]; infer_eltypes=true))

xl = XLSX.readxlsx("data/windscenarios.xlsx")
windscenarios = XLSX.sheetnames(xl)
data_lina["windscenarios"] = DataFrame(XLSX.gettable(xl["windscenarios"]; infer_eltypes=true))
print("data read in ready")

data read in ready

### Set generation

In [8]:
G = data_lina["generatorinfo"][:, :ID]
L = data_lina["lineconstraints"][:, :ID]
T = data_lina["initdemand"][:, :t]          #times
N = data_lina["nodeinfo"][:, :ID]
S = unique(data_lina["windscenarios"][:, :node])
K = unique(data_lina["curtailmentinfo"][:, :k])

NG = length(G)
NL = length(L)
NT = length(T)
NN = length(N)
NS = length(S)
NK = length(K)

3

In [9]:
#generatorinfo
gd = Dict()
for c in names(data_lina["generatorinfo"])
    arr = AxisArray(data_lina["generatorinfo"][:, c], g=G)
    gd[c] = arr
end

#lineconstraints
ld = Dict(c => AxisArray(data_lina["lineconstraints"][:, c], l=L) for c in names(data_lina["lineconstraints"]));

B = AxisArray(zeros(NL, NN), l=L, n=N)
for l in L
    from_node = ld["from"][l]
    to_node = ld["to "][l]
    X = ld["X"][l]
    for (idn, node) in enumerate(N)
        if node == from_node
            B[l, idn] = -1/X
        elseif node == to_node
            B[l, idn] = 1/X
        end
    end
end 

#initdemand
load = AxisArray(Matrix(data_lina["initdemand"][:, Not(:t)]), t=T, n=N);
cost_ls = 100.0

#shiftinginfo
cols = names(data_lina["shiftinginfo"][:, Not(["t", "n", "k"])])

DRCd = Dict(c => AxisArray(zeros(NT, NN, NK), t=T, n=N, k=K) for c in cols)

for r in eachrow(data_lina["shiftinginfo"])
    for c in cols
        DRCd[c][r[:t], r[:n], r[:k]] = r[c]
    end
end
max_rec = 2.0

#curtailmentinfo
cols = names(data_lina["curtailmentinfo"][:, Not(["t", "n", "k"])])

DRSd = Dict(c => AxisArray(zeros(NT, NN, NK), t=T, n=N, k=K) for c in cols)

for r in eachrow(data_lina["curtailmentinfo"])
    for c in cols
        DRSd[c][r[:t], r[:n], r[:k]] = r[c]
    end
end

#location
ng = AxisArray(
    [[g for g in G if gd["location"][g] == idn] for (idn, n) in enumerate(N)],
    n = N
)

#node
nl = AxisArray(zeros(Int, NN, NL), n=N, l=L)
for l in L
    nl[N[ld["from"][l]], l] = 1
    nl[N[ld["to "][l]], l] = -1
end

# Model

In [19]:
bdr = Model(CPLEX.Optimizer)

@variables(bdr, begin
    pgen[G, T] >= 0 # scheduled gen. output
    u[G, T], Bin # state of generator
    θ0[N, T] # phase angles in scheduling phase
    θ1[N, S, T] # phase angles in balancing phase
    Wsc[N, T] >= 0# scheduled wind power
    r_up[G, T] >= 0 # up regulation
    r_down[G, T] >= 0 # down regulation
    r_ups[S, G, T] >= 0 # actual up regulation
    r_downs[S, G, T] >= 0 # actual down regulation 
    Lshed[S, N, T] >= 0 # load shedding
    Wspill[S, N, T] >= 0 # wind spillage
    suc[1:NT, G] >= 0 # generator start-up/shut-down cost
    # DR agg <> ISO contract parameters
    DRCK[T, N, K] >= 0 # DR curtailment blocks
    DRSK[T, N, K] >= 0 # DR shifting blocks
    DRC_SU[T, N, K] >= 0 # DRC startup cost
    DRSRK[T, N, K] >= 0 # DR recovery blocks
    uDRC[T, N, K], Bin # DR curtailment active
    uDRS[T, N, K], Bin # DR shifting activate
    yDR[T, N, K], Bin # DRC activated
    zDR[T, N, K], Bin # DRC de-activated
    uDRSR[T, N, K], Bin # DRS recovery active
    # same for DR agg <> cust. contracts
    DBCK[T, N, K] >= 0
    DBSK[T, N, K] >= 0
    DBC_SU[T, N, K] >= 0 
    DBSRK[T, N, K] >= 0
    uDBC[T, N, K], Bin
    uDBS[T, N, K], Bin
    yDB[T, N, K], Bin
    zDB[T, N, K], Bin
    uDBSR[T, N, K], Bin
    # Dual multipliers/linearization variables for KKT formulation
    A1[T,N,K] >= 0 
    B1[T,N,K] >= 0
    D1[T,N,K] >= 0
    E1[T,N,K] >= 0
    F1[T,N,K] >= 0
    H1[T,N,K] >= 0
    I1[T,N,K] >= 0
    K1[T,N,K] >= 0
    L1[T,N,K] >= 0
    M1[T,N,K] >= 0
end)

# fix slack bus
fix.(θ0["n1", :], 0);
fix.(θ1["n1", :, :], 0);

@expressions(bdr, begin
    # DR costs/profits
    DRC[t in T, n in N], sum(DRCd["cost"][t,n,k]*DRCK[t, n, k]+DRC_SU[t,n,k] for k in K)
    DRS[t in T, n in N], sum(DRSd["cost"][t,n,k]*DRSK[t, n, k] for k in K)
    DBC[t in T, n in N], sum(DRCd["costb"][t,n,k]*DBCK[t, n, k] for k in K)
    DBS[t in T, n in N], sum(DRSd["costb"][t,n,k]*DBSK[t, n, k] for k in K)
    pflow0[l in L, t in T], sum(B[l,n]*θ0[n,t] for n in N)
    pflow1[l in L, s in S, t in T], sum(B[l,n]*θ1[n,s,t] for n in N)
    totalcost, 
        sum(
            sum(
                pgen[g,t] * gd["cost"][g] +
                r_up[g,t] * gd["reserupcost"][g] +
                r_down[g,t] * gd["reserdncost"][g] +
                suc[findfirst(isequal(t), T),g]
            for g in G) +
            sum(DRC[t, n] + DRS[t, n] for n in N) +
            sum(
                pi[s] * (
                    sum(gd["cost"][g] * (r_ups[s,g,t]-r_downs[s,g,t]) for g in G) +
                    sum(cost_ws * Wspill[s,n,t] + cost_ls * Lshed[s,n,t] for n in N)
                    )
            for s in S)
        for t in T)
end)

@constraints(bdr, begin
    # generator limits
    pup[g in G, t in T], pgen[g, t] + r_up[g, t] <= gd["maxcap"][g] * u[g,t]
    pdn[g in G, t in T], pgen[g, t] - r_down[g, t] >= gd["mincap"][g] * u[g,t]
    res_up[g in G, t in T], r_up[g, t] <= gd["reserup"][g]
    res_dn[g in G, t in T], r_down[g, t] <= gd["reserdn"][g]
    startcost[g in G, τ in 1:NT], suc[τ,g] >= gd["startup"][g] * (u[g,T[τ]] - u[g,T[max(τ-1,1)]]) 
    shutcost[g in G, τ in 1:NT], suc[τ,g] >= gd["shutdown"][g] * (u[g,T[max(τ-1,1)]] - u[g,T[τ]])
    rampup[g in G, τ in 2:NT], pgen[g, T[τ]] - pgen[g, T[τ-1]] <= gd["ramp"][g]    
    rampdn[g in G, τ in 2:NT], pgen[g, T[τ-1]] - pgen[g, T[τ]] <= gd["ramp"][g]
    # network flows and balance
    bal0[n in N, t in T], 
        sum(pgen[g,t] for g in ng[n]) + 
        Wsc[n,t] + 
        sum(DRCK[t,n,k] + DRSK[t,n,k] - DRSRK[t,n,k] for k in K) == 
        load[t,n] + sum(pflow0[l, t]*nl[n,l] for l in L)
    pfmax0[l in L, t in T], pflow0[l, t] <= ld["flowcap"][l]
    pfmin0[l in L, t in T], pflow0[l, t] >= -ld["flowcap"][l]
    # limit on wind scheduling
    windcap[n in N, t in T], Wsc[n, t] <= ewind[n, t]
    # DR curtailment constraints (ISO <> DR agg.)
    drcmax[t in T, n in N, k in K], DRCK[t,n,k] <= uDRC[t,n,k] * DRCd["max"][t,n,k]
    drcmin[t in T, n in N, k in K], DRCK[t,n,k] >= uDRC[t,n,k] * DRCd["min"][t,n,k]
    drcst1[τ in 2:NT, n in N, k in K], yDR[T[τ],n,k] - zDR[T[τ],n,k] == uDRC[T[τ],n,k] - uDRC[T[τ-1],n,k]
    drcst2[t in T, n in N, k in K], yDR[t,n,k] + zDR[t,n,k] <= 1
    drcst3[t in T, n in N, k in K], DRC_SU[t,n,k] >= DRCd["SUC"][t,n,k] * yDR[t,n,k]
    # DR shifting constraints (ISO <> DR agg.)
    drsmax[t in T, n in N, k in K], DRSK[t,n,k] <= uDRS[t,n,k] * DRSd["max"][t,n,k]
    drsmin[t in T, n in N, k in K], DRSK[t,n,k] >= uDRS[t,n,k] * DRSd["min"][t,n,k]
    drsmxr[t in T, n in N, k in K], DRSRK[t,n,k] <= max_rec*uDRSR[t,n,k]
    drsorr[t in T, n in N, k in K], uDRS[t,n,k] + uDRSR[t,n,k] <= 1
    drbbal[n in N, k in K], sum(DRSRK[t,n,k] for t in T) == sum(DRSK[t,n,k] for t in T)
    # DR curt. cons (DR agg. <> cust.)
    dbcmax[t in T, n in N, k in K], DBCK[t,n,k] <= uDRC[t,n,k] * DRCd["max"][t,n,k]
    dbcmin[t in T, n in N, k in K], DBCK[t,n,k] >= uDRC[t,n,k] * DRCd["min"][t,n,k]
    dbcst1[τ in 2:NT, n in N, k in K], yDB[T[τ],n,k] - zDB[T[τ],n,k] == uDBC[T[τ],n,k] - uDBC[T[τ-1],n,k]
    dbcst2[t in T, n in N, k in K], yDB[t,n,k] + zDB[t,n,k] <= 1
    dbcst3[t in T, n in N, k in K], DBC_SU[t,n,k] >= DRCd["SUCb"][t,n,k] * yDB[t,n,k]
    # DR shift. cons (DR agg. <> cust.)
    dbsmax[t in T, n in N, k in K], DBSK[t,n,k] <= uDBS[t,n,k] * DRSd["max"][t,n,k]
    dbsmin[t in T, n in N, k in K], DBSK[t,n,k] >= uDBS[t,n,k] * DRSd["min"][t,n,k]
    dbsmxr[t in T, n in N, k in K], DBSRK[t,n,k] <= max_rec*uDBSR[t,n,k]
    dbsorr[t in T, n in N, k in K], uDBS[t,n,k] + uDBSR[t,n,k] <= 1
    dbbbal[n in N, k in K], sum(DBSRK[t,n,k] for t in T) == sum(DBSK[t,n,k] for t in T)
    # Dual multipliers/linearization constraints
    conAB[t in T, n in N, k in K], A1[t,n,k]-B1[t,n,k] >= DRCd["cost"][t,n,k]
    conDE[t in T, n in N, k in K], D1[t,n,k]-E1[t,n,k] >= DRSd["cost"][t,n,k]
    conHI[t in T, n in N, k in K], H1[t,n,k]-I1[t,n,k] >= -DRCd["costb"][t,n,k]
    conKL[t in T, n in N, k in K], K1[t,n,k]-L1[t,n,k] >= -DRSd["costb"][t,n,k]
    strong_duality, 
        sum(
            A1[t,n,k]*DRCd["max"][t,n,k] - B1[t,n,k]*DRCd["min"][t,n,k] +
            D1[t,n,k]*DRSd["max"][t,n,k] - E1[t,n,k]*DRSd["min"][t,n,k] +
            F1[t,n,k]*max_rec +
            H1[t,n,k]*DRCd["max"][t,n,k] - I1[t,n,k]*DRCd["min"][t,n,k] +
            K1[t,n,k]*DRSd["max"][t,n,k] - L1[t,n,k]*DRSd["min"][t,n,k] +
            M1[t,n,k]*max_rec
        for t in T for n in N for k in K) ==
        sum(
            DRCd["cost"][t,n,k] * DRCK[t,n,k] +
            DRSd["cost"][t,n,k] * DRSK[t,n,k] - 
            DRCd["costb"][t,n,k] * DBCK[t,n,k] - 
            DRSd["costb"][t,n,k] * DBSK[t,n,k]    
        for t in T for n in N for k in K)
    # second stage (after wind realizes)
    resup[s in S, g in G, t in T], r_ups[s,g,t] <= r_up[g,t]
    resdn[s in S, g in G, t in T], r_downs[s,g,t] <= r_down[g,t]
    bals[s in S, n in N, t in T], 
        sum(r_ups[s,g,t]-r_downs[s,g,t] for g in ng[n]) + 
        wind[s,n,t] - Wsc[n,t] - Wspill[s,n,t] + Lshed[s,n,t] == 
        -sum((pflow1[l,s,t] - pflow0[l,t])*nl[n,l] for l in L)
    windspill[s in S, n in N, t in T], Wspill[s,n,t] <= wind[s,n,t]
    pfmax1[l in L, s in S, t in T], pflow1[l,s,t] <= ld["flowcap"][l]
    pfmin1[l in L, s in S, t in T], pflow1[l,s,t] >= -ld["flowcap"][l]
end)
    
@objective(bdr, Min, totalcost)

bdr

UndefVarError: UndefVarError: CPLEX not defined

In [20]:
optimize!(bdr)

UndefVarError: UndefVarError: bdr not defined

In [21]:
termination_status(bdr)

UndefVarError: UndefVarError: bdr not defined

In [22]:
objective_value(bdr)

UndefVarError: UndefVarError: bdr not defined

In [23]:
value.(pgen)

UndefVarError: UndefVarError: pgen not defined

In [24]:
value.(u)

UndefVarError: UndefVarError: u not defined

In [25]:
plot(
    sum(load, dims=2)[:, 1], 
    label="Load before DR",
    xlims=(1,24),
    xticks=6:6:24,
    )

plot!(
    [sum(load[t,:]) - sum(value.(DRCK[t,:,:]) .+ value.(DRSK[t,:,:]) .- value.(DRSRK[t,:,:])) for t in T], 
    label="Load after DR"
    )

UndefVarError: UndefVarError: load not defined

In [26]:
plot(
    ewind["n6", :], 
    ribbon = stdwind["n6", :],
    label="μw +- σw",
    xlims=(1,24),
    xticks=6:6:24,
    )

UndefVarError: UndefVarError: ewind not defined